In [2]:
import pandas as pd
import spacy
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from xgboost import XGBRegressor  # Replace this with any other model you're testing
from sklearn.multioutput import MultiOutputRegressor
from transformers import LongformerTokenizer, LongformerModel
import torch

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
nlp = spacy.load("en_core_web_sm")

In [5]:
dataset = pd.read_excel('combined_hvp_numeric.xlsx')

In [6]:
Messages = dataset['Message']

In [7]:
Messages

0      I can morning. i'm Okay, how are you. that's a...
1      i'm doing good give me a. Second. yeah I can h...
2      morning. Barely good one are barely. that's al...
3      How are you. How. A lot a lot Oh, she got the ...
4      yeah I went to that page and I don't know what...
                             ...                        
653    Hello. Good how are you. Not much has happened...
654    There we go. i'm all right, I think I need to ...
655    Can you hear me. I know just confused I. yeah ...
656    Hello it's morning. You thought for sure I was...
657    around like the first week of September I star...
Name: Message, Length: 658, dtype: object

In [8]:
Traits = dataset.drop(columns=['Message','ID'])

In [9]:
Traits.head()

,AI% 1,AI% 2,BQa1,BQa2,BQr1,BQr2,CQ1,VQ Total +,VQ Total -,SQ Total +,...,DIM-I+ 1 Transposition,DIM-I+ 2 Transposition,DIM-I- 1 Transposition,DIM-I- 2 Transposition,DIM-S+ 1 Transposition,DIM-S+ 2 Transposition,DIM-S- 1 Transposition,DIM-S- 2 Transposition,Rho 1,Rho 2
0,0.500000,0.864583,151,90,6.9,13.9,1042,13,13,13,...,1,0,6,29,3,1,0,1,0.934,0.133
1,0.500000,0.583333,78,34,1.6,1.1,125,14,14,25,...,0,0,2,10,3,7,4,5,0.905,0.695
2,0.600000,0.702381,103,46,3.3,4.1,340,12,18,25,...,1,5,1,10,0,4,1,9,0.895,0.364
3,0.634615,0.500000,80,30,0.8,0.6,64,19,33,24,...,1,2,5,6,2,3,6,2,0.754,0.802
4,0.611111,0.500000,66,24,1.4,1.7,92,14,22,24,...,3,3,2,2,1,8,3,2,0.897,0.812


In [10]:
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return tokens

In [11]:
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
model = LongformerModel.from_pretrained("allenai/longformer-base-4096")

def tokens_to_embedding(tokens):
    sentence = " ".join(tokens)
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    token_embeddings = outputs.last_hidden_state
    sentence_embedding = token_embeddings.mean(dim=1).squeeze()
    return sentence_embedding.numpy()

In [12]:
# Messages = Messages.apply(preprocess_text)

In [13]:
# embedded_messages = Messages.copy(deep=True)

In [14]:
# embedded_messages = [list(set(tokens)) for tokens in embedded_messages]

In [15]:
# embedded_messages = [tokens_to_bert_embedding(tokens) for tokens in embedded_messages]

In [16]:
# np.save('embeddings_hvp.npy', np.array(embedded_messages))

In [17]:
# x = np.array(embedded_messages)

In [18]:
y = Traits.values
scaler = MinMaxScaler()
y = scaler.fit_transform(y)

In [19]:
y.shape

(658, 112)

In [20]:
# np.save('output_training_vec_hvp.npy', y)

# Loading Saved Embeddings

In [21]:
x = np.load('embeddings_hvp.npy')

In [22]:
x.shape

(658, 768)

In [23]:
# y = np.load('/content/drive/MyDrive/Copy of output_training_vec_hvp.npy')

In [24]:
# y.shape

In [25]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.optimizers import Adam
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error


# nn = Sequential([
#     Dense(512, input_shape=(768,), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     Dropout(0.3),
#     Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     Dropout(0.3),
#     Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     Dense(112)
# ])


# nn.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# history = nn.fit(x, y, epochs=100, batch_size=16, validation_split=0.2, verbose=1)

In [26]:
# results = nn.evaluate(x, y)
# print('MAE: %.3f (%.3f)' % (np.mean(results), np.std(results)))

In [27]:
# import matplotlib.pyplot as plt
# plt.plot(history.history['loss'], label='Training Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Model Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [28]:
# Messages

In [29]:
# msg = Messages[0]
# msg

In [30]:
# msg = preprocess_text(msg)

In [31]:
# msg = list(set(msg))

In [32]:
# msg = tokens_to_embedding(msg)

In [33]:
# msg = msg.reshape(1,-1)

In [34]:
# msg.shape

In [35]:
# y_out = nn.predict(msg)

In [36]:
# y_out.shape

In [37]:
# y_actual = y[0].reshape(1,-1)

In [38]:
# y_actual.shape

In [39]:
# y_out = scaler.inverse_transform(y_out)
# y_actual = scaler.inverse_transform(y_actual)

In [40]:
# nn.evaluate(x,y)

In [41]:
# r2_score(y_actual[0],y_out[0])

In [42]:
# Define the XGBoost model wrapped in MultiOutputRegressor for multi-output regression
xgb_model = MultiOutputRegressor(XGBRegressor(objective="reg:squarederror"))

# Train the model on the entire dataset
xgb_model.fit(x, y)


MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...))

In [43]:
def get_predictions(msg):
    msg = preprocess_text(msg)
    msg = list(set(msg))
    msg = tokens_to_embedding(msg)
    msg = msg.reshape(1,-1)
    y_out = xgb_model.predict(msg)
    return scaler.inverse_transform(y_out)

In [86]:
output = []
for i,row in dataset.iterrows():
    msg = row['Message']
    y_actual = np.array(row.drop(['Message', 'ID']))
    y_actual = y_actual.reshape(1,-1)
    y_pred = get_predictions(msg)
    predicted_with = r2_score(y_actual[0],y_pred[0])
    inl = [row['ID']] + [round(float(pred_scr),3) for pred_scr in list(y_pred[0])] + [predicted_with]
    output.append(inl)



In [64]:
d_cols = list(dataset.drop(columns=['Message','ID']).columns)
output_columns  = ['ID']+[ f'pred_{column}' for column in d_cols] + ['predicted_with']

In [65]:
output_df = pd.DataFrame(output,columns=output_columns)

In [66]:
output_df.to_excel('hvp_predictions.xlsx',index=False)

In [67]:
output_cols = "AI% 1,AI% 2,BQr1,BQr2,VQ Total +,DI 1,DIM-E 1 Bias Score,DIM-E 2 Bias Score,DIM-I 2 Bias Score,DIM-I 1 Bias Score,DIM-S 1 Bias Score,DIM-S 2 Bias Score,DIM-I+ 1 Composition,DIM-I+ 2 Composition,DIM-I- 1 Composition,DIM-I- 2 Composition,DIM-S+ 1 Composition,DIM-S+ 2 Composition,DIM-S- 1 Composition,DIM-S- 2 Composition,INT-E 1 Compositions,INT-E 2 Compositions,INT-I 1 Compositions,INT-I 2 Compositions,INT-S 1 Compositions,INT-S 2 Compositions,DI 2,DIF 2,DIM 1,DIM% 1,DIM-E 1 Valuation Bias Severity,DIM-E 2 Valuation Bias Severity,DIM-E- 2,DIM-I 1,DIM-I 1 Transpositions,DIM-I 1 Compositions,DIM-I 1 Valuation Bias Severity,DIM-I 2,DIM-I 2 Valuation Bias Severity,DIM-I+ 1,DIM-I+ 2,DIM-I- 1,DIM-S 1,DIM-S 1 Transpositions,DIM-S 1 Compositions,DIM-S 1 Valuation Bias Severity,DIM-S 2 Valuation Bias Severity,DIM-S+ 1,DIM-S- 1,DIS 1,INT-I 1,DIM-E+ 1 Composition,DIM-E+ 2 Composition,DIM-E+ 1 Transposition,DIM-E+ 2 Transposition,DIM-E- 1 Transposition,DIM-E- 2 Transposition,DIM-I+ 1 Transposition,DIM-I+ 2 Transposition,DIM-I- 1 Transposition,DIM-I- 2 Transposition,DIM-S+ 1 Transposition,DIM-S+ 2 Transposition,DIM-S- 1 Transposition,Rho 1,Rho 2"
y_columns = [f'pred_{col.strip()}' for col in output_cols.split(",")]

In [87]:
output_df[y_columns]

,pred_AI% 1,pred_AI% 2,pred_BQr1,pred_BQr2,pred_VQ Total +,pred_DI 1,pred_DIM-E 1 Bias Score,pred_DIM-E 2 Bias Score,pred_DIM-I 2 Bias Score,pred_DIM-I 1 Bias Score,...,pred_DIM-E- 2 Transposition,pred_DIM-I+ 1 Transposition,pred_DIM-I+ 2 Transposition,pred_DIM-I- 1 Transposition,pred_DIM-I- 2 Transposition,pred_DIM-S+ 1 Transposition,pred_DIM-S+ 2 Transposition,pred_DIM-S- 1 Transposition,pred_Rho 1,pred_Rho 2
0,0.569,0.751,7.806,6.968,9.378,4.767,-0.145,-0.191,-0.921,-0.267,...,8.349,1.112,0.267,3.837,24.205,2.177,2.774,0.177,0.926,0.272
1,0.582,0.733,4.261,2.225,11.474,14.954,0.054,-0.337,-0.671,0.165,...,9.984,1.336,0.872,6.237,11.476,1.868,3.791,3.290,0.908,0.458
2,0.587,0.811,2.986,49.046,14.354,7.767,-0.628,-0.696,-0.290,0.084,...,9.852,0.420,1.497,1.712,4.622,2.857,3.451,4.006,0.826,0.076
3,0.746,0.693,2.167,3.447,13.288,12.927,-0.148,0.287,-0.580,0.412,...,5.541,2.230,1.825,2.883,16.731,2.414,7.003,6.443,0.805,0.314
4,0.621,0.628,1.822,6.898,12.041,2.408,-0.213,0.460,-0.713,-0.411,...,10.750,1.430,2.059,2.647,9.475,0.608,3.815,2.063,0.918,0.753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,0.664,0.607,2.739,7.369,11.377,11.714,-0.267,0.619,-0.848,-0.190,...,4.009,1.138,0.942,4.282,7.984,-0.029,4.229,1.346,0.918,0.479
654,0.622,0.721,3.809,5.303,9.779,6.309,0.099,-0.415,-0.788,-0.003,...,10.433,1.238,0.545,2.770,10.991,0.586,5.368,1.585,0.896,0.352
655,0.576,0.878,5.829,4.604,10.410,10.721,-0.517,-0.578,-0.912,-0.499,...,17.370,0.577,-0.258,3.989,17.056,0.925,5.637,2.833,0.848,-0.122
656,0.539,0.695,4.476,7.147,9.207,5.015,0.369,-0.660,-0.899,-0.319,...,6.181,1.328,1.658,4.931,19.387,2.867,5.673,0.670,0.886,0.734


In [88]:
output_custom = output_df[y_columns]

In [90]:
output_custom.to_excel('hvp66outputs.xlsx',index=False)